<div class="alert alert-block alert-success">
A green text box indicates a code cell that must be run, without alteration, to complete the workflow.
</div>

<div class="alert alert-block alert-warning">
An orange text box indicates an optional code cell that doesn't have to be run to complete the workflow, but can be run to complete optional tasks.
</div>

<div class="alert alert-block alert-info">
A blue text box indicates a code cell that requires user input - this cell also must be run to complete the workflow, but the user needs to modify the command in the cell.
</div>

<div class="alert alert-block alert-danger">
In addition, some text boxes contain particularly important information. These will be coloured red.
</div>

# <span style="color:green"> Import python functions </sapan>
<div class="alert alert-block alert-success">
    These packages should all be installed and available in your default environment. eResearch can help with installing modules and setting up environments. 
</div>


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

import scipy
from scipy import stats as stats
from copy import copy as copy

from openpyxl import load_workbook
from openpyxl.worksheet.table import Table

# from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering


from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

from sklearn.neighbors import kneighbors_graph


# <span style="color:green"> Define/Import custom functions </sapan>
<div class="alert alert-block alert-success">
    Custom functions for this workflow are imported from the functions folder. 
</div>

In [ ]:
from functions.plotting import(
    volcanoPlot, 
    plot_dendrogram,
    get_factor_colours,
    get_connectivity
    )

In [ ]:
from functions.eda import (get_AggloModel)

# <span style="color:orange"> Configure output options for this run </span>

<div class="alert alert-block alert-warning">
    The writeOutput variable below enables high level control for whether output files are written. This can be turned off to prevent overwriting existing files. <br>
    AutoRunProject allows the selection of a folder location from a projects.txt config file
</div>

In [ ]:
writeOutput = False
# writeOutput = True


## <span style="color:blue"> Read in config file </sapan>
<div class="alert alert-block alert-info">
    If running in autoRunProject mode, a project.txt file is used to hold all current projects, with one project per line. Inactive projects or comment lines start with a #. The project to run must be uncommented.<br>
A config file (config.txt) or project file (project.txt) must be present in the folder given in the projects file or can be entered in the text below.
</div>

In [ ]:
autoRunProject = True
autoRunProject = False

if autoRunProject:
    try:
        with open('projects.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if ((not line.startswith('#')) and (not line.strip()=='')):
                    subfolder = line
    except FileNotFoundError:
        subfolder = input("Project file not found. Enter the name of the config folder and press enter (Must be same level as code folder)")
        # print('error')
        
else:
    subfolder = input("Enter the name of the working folder (Must be same level as code folder)")

print(subfolder)

os.chdir("../" + subfolder)
# os.getcwd()

In [ ]:
# Show current working directory to confirm that directory has been changed
os.getcwd()

In [ ]:
# read in paths from config file
configDict = {
    'rootDir': '',
    'initialDataPath' : '',
    'QCDataPath' : '',
    'labWorksheet01Path':'',
    'sampleInfoFile' : '',
    'projectName':'',
    'selectedData': []
}

with open('./config.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith('#'):
            line = line.strip()
            fields = line.split(':')

            if fields[0].strip()=='initialDataPath':
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
            elif fields[0].strip()=='probeThresholdIdx':
                configDict[fields[0].strip()] = int(fields[1].strip().strip('\''))
            elif fields[0].strip()=='selectedData':
                tempList = fields[1].strip().strip('\'').split(',')
                tempList = [x.strip() for x in tempList]
                configDict['selectedData'] = tempList
            else:
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
## ToDo: Add checks to ensure that minimal fields have been populated. Raise errors or warnings

configDict

# <span style="color:green"> Run Analysis </sapan>
<div class="alert alert-block alert-success">
    
</div>

In [ ]:
# ToDo:
# add sample info file details into config file in previous notebook
# sampleInfo_with_Wells.csv

In [ ]:
# ToDo: Should initial filtering be done using none-mean-HKGeoMean normalised data. Using a type of background subtraction for filtering would bias towards removing noisy or low expressing samples. Removing these may give more accurate estimates of real probe values.

## <span style="color:green"> Read-in nanostring normalised data </sapan>
<div class="alert alert-block alert-success">
    Read in normalised data from the final step of the QC notebook. This includes many versions of normalised data for comparison to help choose the final normalisation approach.
</div>

In [ ]:
normDir = os.path.join(configDict['rootDir'], 'Normalisation')
# print(normDir)
QCDataFile = [f for f in os.listdir(normDir) if (f.startswith('QC') and f.endswith('_preNorm.csv'))][0]
print(QCDataFile)


In [ ]:
os.listdir(normDir)

### <span style="color:green"> Sort samples and probes  </sapan>
<div class="alert alert-block alert-success">
    Read and save the probe order and sample order from the pre normalisation data set for consistent organisation of data in plots.
</div>

In [ ]:
QCDataDF = pd.read_csv(os.path.join(normDir, QCDataFile), index_col=0)
# QCDataDF.columns = [x.replace(' ','.') for x in QCDataDF.columns]
# QCDataDF.columns = [x.replace('-','.') for x in QCDataDF.columns]

codeClass = QCDataDF.loc[:,'Code.Class']

probeOrder = QCDataDF.index[8:]
sampleOrder = QCDataDF.columns[1:]

# sampleOrder

In [ ]:
QCDataDF

In [ ]:
# QCDataDF.columns = [x.replace(' ','.') for x in QCDataDF.columns]
# QCDataDF.columns = [x.replace('#','.') for x in QCDataDF.columns]
# QCDataDF.columns = [x.replace('/','.') for x in QCDataDF.columns]

## <span style="color:green"> Read-in sampleInfo </sapan>
<div class="alert alert-block alert-success">
Read in sample information from sampleInfo_with_Wells.csv file created in the QC notebook.
</div>

In [ ]:
sampleInfo = pd.read_csv(os.path.join(configDict['rootDir'], configDict['sampleInfoFile']), index_col=0)

In [ ]:
colTemp = sampleInfo.columns
# colTemp = [x.replace(' ','.') for x in colTemp]
# colTemp = [x.replace('-','.') for x in colTemp]
sampleInfo.columns = colTemp


sampleInfo = sampleInfo.loc[:,sampleOrder]

In [ ]:
# sampleInfo.columns = [x.replace(' ','.') for x in sampleInfo.columns]
# sampleInfo.columns = [x.replace('#','.') for x in sampleInfo.columns]
# sampleInfo.columns = [x.replace('/','.') for x in sampleInfo.columns]

In [ ]:
sampleInfo

## <span style="color:green"> Read-in factors for use in data grouping from config file. </sapan>
<div class="alert alert-block alert-success">
Read in factors of interest as selected in QC notebook (reads from factor_lookup.tsv file).
</div>

In [ ]:
# factors = input('enter factors to be used for groups to check probe expression. separate multiple factors by a comma')
factors = configDict['selectedData']


In [ ]:
# generate a dictionary to hold factors for group analysis
factorDict = {}
factorDict2 = {}

for f in factors:
    entries = list(set(sampleInfo.loc[f].values))
    print(f)
    print(entries)
    # print(sorted(entries))
    print()
    factorDict[f] = entries
    factorDict2[f] = {}
    for e in entries:
        factorDict2[f][e] = sampleInfo.columns[sampleInfo.loc[f] == e]
    

In [ ]:
# factorDict.keys()

In [ ]:
# factorDict2

# <span style="color:green"> Visualise Nanostring Norm results and choose samples to be kept for final normalisation </sapan>
<div class="alert alert-block alert-success">
The first normalisation step is to view all normalisation options and select a relatively harsh/conservative method (e.g. with background subtraction) to run thresholding of probes and samples.</div>

In [ ]:
# configDict

In [ ]:
# QCDataDF

In [ ]:
files = os.listdir(os.path.join(normDir, 'NSNorm'))
files = sorted(files)
print(f'number of files found :\t{len(files)}')

In [ ]:
# probeOrder

In [ ]:
# sampleOrder = [x.replace(' ','.') for x in sampleOrder]
# sampleOrder = [x.replace('#','.') for x in sampleOrder]
# sampleOrder = [x.replace('/','.') for x in sampleOrder]

In [ ]:
width = 6
height = 14
height = 11

fig, axs = plt.subplots(height, width, figsize=[40,25])
# fig.suptitle('Nanostring Normalisation heatmaps')

for y in range(height):
    for x in range(width):
        fileIdx = x + y*width
        tempDF = pd.read_csv(os.path.join(normDir, 'NSNorm',files[fileIdx]), index_col=0)
#         axs[y][x].matshow(np.log2(tempDF + 1), aspect = 'auto', cmap='coolwarm')
        axs[y][x].matshow(np.log2(tempDF.loc[probeOrder,sampleOrder] + 1), cmap='coolwarm')
        axs[y][x].set_xticks([])
        axs[y][x].set_yticks([])
        axs[y][x].set_title(files[fileIdx][:-4])
    
# plt.tight_layout()
# fig.show()
fig.savefig('NSNorm.png')


## <span style="color:red"> What to look fo in the plots above </sapan>

<div class="alert alert-block alert-danger">

Naming convention is extraERCC_BackgroundSubtraction_NormalisingFactor


Plots are laid out in groups of 12, being 6 plots across and 2 plots down. Each set of 12 has the same normalising factor applies ( in order, these are ___,___,HKGeoMean,LowCVGeoMean,___).

At this step we want to choose a method where low expressers (likely to be at or below the limit of detection) are set to a very low (or 0) value (dark blue). A safe choice is usually None_Mean_HKGeoMean (list index 27)
</div>

# <span style="color:green"> Threshold data </sapan>
<div class="alert alert-block alert-success">
Here we start to perform the final data thresholding before differential analysis.
</div>

In [ ]:
#define control probe names
#ToDo: Make this robust for Mouse assays also.

controlSet = set(['HYB-NEG', 'HYB-POS', 'Rb IgG', 'Ms IgG2a', 'Ms IgG1'])

### <span style="color:green"> Threshold probes and drop low or null expressing probes </sapan>
<div class="alert alert-block alert-success">
    Description here

The following variables can be set by the user, and the default values are as follows.

threshold = 1.5
expPropCutOff = 0.5
SamplePropCutOff = 0.5

Threshold is the probe value to define the limit of detection. With background subtraction, a relatively low value works well. A higher value will likely be needed if a normalised data set without background correction is used for this step.
expPropCutOff: Expressed Proportion cutoff 
SamplePropCutOff: Sample Proportion Cutoff

</div>

In [ ]:
# ToDo: Add in a variable called  thresholdData to hold data for comparison at thresholding stage. This will be read from one of the normalised files.

# ToDo: Examine which normalised data should be used for thresholding. Does it make a difference if background correction has been performed?

# ToDo: Read norm Method from config file

In [ ]:
# thresholdData = pd.read_csv(os.path.join(normDir, 'NSNorm',files[51]), index_col=0) # none_mean_lowCVGeoMean
# thresholdData = pd.read_csv(os.path.join(normDir, 'NSNorm',files[24]), index_col=0) #None_None_HKGeoMean

thresholdData = pd.read_csv(os.path.join(normDir, 'NSNorm',files[27]), index_col=0) #None_Mean_HKGeoMean


thresholdData.columns = [x.replace(' ','.') for x in thresholdData.columns]
thresholdData.columns = [x.replace('-','.') for x in thresholdData.columns]



In [ ]:
# ToDo: Check that thresholding is not too restrictive. Should allow probes to be kept if they are expressed in more than half of any sub group.

# ToDo: don't look for low expressing probes in groups with only 1 or 2 samples.

In [ ]:
threshold = 1.5
expPropCutOff = 0.5
SamplePropCutOff = 0.5

dropList = []
dropSetTemp = []

for f in factorDict2.keys():
    print(f)
    for g in factorDict2[f].keys():
        # print(g)
        # print(factorDict2[f][g])
        groupLen = len(factorDict2[f][g])
        # print(groupLen)
        passThreshold = (thresholdData[factorDict2[f][g]]>threshold).sum(axis=1)
        # print(passThreshold)
        passThreshProp = (passThreshold/groupLen) < expPropCutOff
        # print(passThreshProp)
        failIdx = thresholdData.index[passThreshProp]
        # print(failIdx)
        # print(len(failIdx))

        if (len(failIdx) > 0):
            dropList.extend(list(failIdx))
            dropSetTemp.append(list(failIdx))
        # print(thresholdData.index[((thresholdData[factorDict2[f][g]]>2).sum(axis=1)/groupLen) < 0.5])

dropList = list(set(dropList) - controlSet)
print(dropList)
print(dropSetTemp)

# dropSet = set(tuple(x) for x in dropSetTemp)

In [ ]:
try:
    dropSet = set(dropSetTemp[0])
    hasDrops = True
except IndexError:
    hasDrops = False

if hasDrops:
    for x in range(1,len(dropSetTemp)):
        # print(x)
        dropSet = dropSet & set(dropSetTemp[1])
    
    dropSet = dropSet - controlSet
    # dropSet
    dropList = list(dropSet)
    # dropList
    QCDataDF = QCDataDF.drop(index=dropList)
    
    
    codeClass = codeClass[QCDataDF.index]
    print(codeClass)
    
    
    print(len(dropList))
    print(dropList)


### <span style="color:green"> Threshold samples and drop low or null expressing samples </sapan>
<div class="alert alert-block alert-success">
    Description here
</div>

In [ ]:
dropSamples = list(QCDataDF[QCDataDF.columns[1:]].T[(QCDataDF[QCDataDF.columns[1:]]>threshold).sum(axis=0)/len(QCDataDF.index) < SamplePropCutOff].index)
print(f'dropSamples :\t{dropSamples}')
QCDataDF = QCDataDF.drop(labels=dropSamples, axis=1)

QCDataDF.index.name = 'Name'
QCDataDF

### <span style="color:green"> Export data after dropping low expressing probes and samples </sapan>
<div class="alert alert-block alert-success">
    Description here
</div>

In [ ]:
# export data after dropping probes and samples due to low expression

project = configDict['projectName']

qcDropCSV = 'QC_' + project + '_preNorm_Dropped.csv'

writeOutput = True
if writeOutput:
    QCDataDF.to_csv(os.path.join(normDir, qcDropCSV))
writeOutput= False

## <span style="color:green"> re-run NS norm with dropped data </sapan>
<div class="alert alert-block alert-success">
    Description here
</div>

In [ ]:
# re-run NSNorm

cmd = 'Rscript ../DSP_EDA_Protein/NSNorm.R -d ' + normDir + ' -s NSNormDropped -f ' + qcDropCSV
print(cmd)
os.system(cmd)

In [ ]:
time.sleep(10)

## <span style="color:green"> View normalised-dropped data </sapan>
<div class="alert alert-block alert-success">
    Description here
</div>

In [ ]:
# Continue with viewing data and QC of normalised data

files = os.listdir(os.path.join(normDir, 'NSNormDropped'))
files = sorted(files)


In [ ]:
width = 6
height = 14

fig, axs = plt.subplots(height, width, figsize=[40,25])
fig.suptitle('Nanostring Normalisation heatmaps')
for y in range(height):
    for x in range(width):
        fileIdx = x + y*width
        tempDF = pd.read_csv(os.path.join(normDir, 'NSNormDropped',files[fileIdx]), index_col=0)
        axs[y][x].matshow(np.log2(tempDF + 1), cmap='coolwarm')
        axs[y][x].set_xticks([])
        axs[y][x].set_yticks([])
        axs[y][x].set_title(files[fileIdx][:-4])
    
# plt.tight_layout()
# fig.show()
fig.savefig('NSNormDropped.png')

In [ ]:
# ToDo: Need function to drop samples from sample info file

# <span style="color:green"> generate groups for EdgeR analysis </sapan>

<div class="alert alert-block alert-danger">
In addition, some text boxes contain particularly important information. These will be coloured red.
</div>

In [ ]:
# temp = noneMeanHKDF > 0 

In [ ]:
# noneMeanHKDF = pd.read_csv(os.path.join(normDir, 'NSNorm',files[27]), index_col=0)
noneMeanHKDF = pd.read_csv(os.path.join(normDir, 'NSNormDropped',files[27]), index_col=0)


In [ ]:
# noneMeanHKDF

In [ ]:
# groupedExpressedIndex = noneMeanHKDF.loc[probeOrder].loc[((noneMeanHKDF > 0 ).sum(axis = 1) / len(noneMeanHKDF.columns) > 0.33333)].index


In [ ]:
# groupedExpressedIndex

In [ ]:
# len(groupedExpressedIndex)

In [ ]:
QCData = pd.read_csv(os.path.join(normDir, QCDataFile), index_col=0)

QCData

# Run EdgeR analysis

Write comparisons to a text file that will be parsed by the r script

In form of factor.variable, factor.variable2, comparisonName
1 comparison per line

### extract potential factors for analysis from info file

### extract potential groups based on factors

### Show group numbers for each of the comparisons

### Set up config file for EdgeR 

#### (Use helper notebook or uncomment code below)

In [ ]:
# # groups = ['Broad_classification']
# comps = [
#     []
# ]

# compNames = []
# for g in comps:
#     ctemp = []
#     for c in g:
#         c = c.replace('.','_')
#         c = c.replace(' - ','_vs_')
#         ctemp.append(c)
#     compNames.append(ctemp)

# compNames

In [ ]:
# outFile = 'EdgeR_Config.txt'

# outLines = []
# for g in range(len(groups)):
#     groupLine = 'GROUP:' + groups[g]
#     outLines.append(groupLine)
    
#     compLine = 'COMPARISON:'
#     compNameLine = 'COMP_NAME:'
#     for c in range(len(comps[g])):
#         compLine += comps[g][c]
#         compLine += ','
#         compNameLine += compNames[g][c]
#         compNameLine += ','
    
#     outLines.append(compLine)
#     outLines.append(compNameLine)

# with open(outFile, 'w') as o:
#     o.write('\n'.join(outLines))
#     o.write('\n')

In [ ]:
# configDict['rootDir']

In [ ]:
# ToDo: Ensure that EdgeR_config.txt has been populated

In [ ]:
# Choose normalisation file for use with edgeR

normPath = os.path.join('Normalisation','NSNormDropped')
# normPath = os.path.join('Normalisation','NSNorm')
print(normPath)

# normFile = 'NanoStringNorm_52_none_mean_low.cv.geo.mean.csv'
# normFile = 'NanoStringNorm_49_none_none_low.cv.geo.mean.csv'
normFile = 'NanoStringNorm_25_none_none_housekeeping.geo.mean.csv'
# normFile = 'NanoStringNorm_13_none_none_housekeeping.sum.csv'
print(normFile)



In [ ]:
# Define sample info file, export directory and 

sampleinfoFile = 'sampleInfo_with_wells.csv'

runname = 'NS' + normFile[10:17]
exportdir = os.path.join('EdgeR', runname)
print(f'exportdir : {exportdir}')


# ensure export directory is created

try:
    os.mkdir(os.path.join(configDict['rootDir'], exportdir))
except FileNotFoundError:
    os.mkdir(os.path.join(configDict['rootDir'], exportdir.split('/')[0]))
except FileExistsError:
    pass
try:
    os.mkdir(os.path.join(configDict['rootDir'], exportdir))
except FileExistsError:
    pass

In [ ]:
## Need to handle dropped samples that may still be present in sampleinfoFile. 
## Could be handled with another file, or by removing samples within R.


In [ ]:
# set up command and run edgeR

cmd = 'Rscript ../DSP_EDA_Protein/EdgeR.R -c ' + os.getcwd()
cmd += ' -d ' + configDict['rootDir']
cmd += ' -n ' + normPath
cmd += ' -f ' + normFile
cmd += ' -e ' + exportdir
cmd += ' -i ' + sampleinfoFile
print(cmd)
os.system(cmd)

# Convert MD Plots to Volcano Plots

In [ ]:
# Get the list of files to run

dataPath = os.path.join(configDict['rootDir'],exportdir,)
filesMaster = []
for root, folder, files in os.walk(dataPath):
    files = [os.path.join(root, f) for f in files if (f.endswith('.csv') and f.startswith('MD_plot'))]
    filesMaster.extend(files)
     

In [ ]:
## Run volcano plots

# sigGenes = []
for file in filesMaster:
    if not (file[-7:-4] == '_tr'):
        subGenes = volcanoPlot(dataPath, file, pVal=False)
        # sigGenes.extend(subGenes)
# sigGenes = list(set(sigGenes))
    

# Working

### Plot heatmaps and dendrograms

In [ ]:
# ToDo: Ensure heatmaps are using relevant normalised data. Read data from NSNorm or EdgeR files

In [ ]:
QCData.drop(labels=['Code.Class'], axis=1)

In [ ]:
## Subset dataframe to contain only significant probes and relevant AOIs

In [ ]:
dendroModel = AgglomerativeClustering(n_clusters=None, 
                                # affinity='euclidean', 
                                # metric='euclidean', 
                                metric='cosine', 
                                memory=None, 
                                connectivity=None, 
                                compute_full_tree=True, 
                                # linkage='ward', 
                                linkage='single', 
                                # linkage='complete', 
                                distance_threshold=0.1, 
                                compute_distances=True
                               )

In [ ]:
# ToDo : Add linkage map from cosine

### Read in  Factor lookup tsv file

In [ ]:
def read_factor_lookup(file):
    factorLookup = {}
    varLookup = {}
    vars = []
    
    sep = '\t'
    if file.lower().endswith('.tsv'):
        sep= '\t'
    elif file.lower().endswith('.csv'):
        sep= ','
    
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            factorVariable = line.split(':')

            # ToDo: CHeck this is correct patter for assertions
            assert len(factorVariable)==2, 'Factor lookup file is incomplete or incorrectly formatted'
            # print(factorVariable)
            factor = factorVariable[0]
            factor = factor.strip()
            # print(factor)
            variables = factorVariable[1].split(sep)
            # print(variables)
            variables = [x.strip() for x in variables]
            vars.extend(variables)
            for var in variables:
                factorLookup[var] = factor

    return(factorLookup, varLookup, vars)


In [ ]:
def get_relevant_AOIs(fileName, factorLookup):
    # Extract subpopulation attributes from file name
    fields = fileName[:-4].split('_')
    fields.remove('MD')
    fields.remove('plot')
    vsIDX = fields.index('vs')
    numIDXs = [x for x in range(vsIDX)]
    denomIDXs = [x for x in range(vsIDX+1,len(fields))]
    AOISuperSets = [[],[]]
    
    for i, IDXs in enumerate([numIDXs,denomIDXs]):
        for idx in IDXs:
            factor = fields[idx]
            print('factor')
            print(factor)
            factorType = factorLookup[factor]
            thisSet = set(sampleInfo.loc[:,sampleInfo.loc[factorType] == factor].columns)
            AOISuperSets[i].append(thisSet)
    numeratorAOIs = set.intersection(*AOISuperSets[0])
    denominatorAOIs = set.intersection(*AOISuperSets[1])     
    relevantAOIs = list(numeratorAOIs | denominatorAOIs)
    relevantAOIs = [x.replace('.',' ') for x in relevantAOIs]
    fields.remove('vs')
    return(relevantAOIs, fields)


In [ ]:
factorLookup, varLookup, vars = read_factor_lookup('factor_lookup.tsv')
factors = list(set(factorLookup.values()))
varLookup = dict(zip(vars, np.arange(0.1,0.9,0.8/len(vars))))


for file in filesMaster:   # Iterate through files


    if not (file[-7:-4] == '_tr'):  # Ignore _tr files with more stringent expression thresholding
        print(file)
        fileName = file.split('/')[-1]
        relevantAOIs, fileNameFields = get_relevant_AOIs(fileName, factorLookup)


        ## Move function to get factor lookup et here so that only relevant factors are used for each plot to simplify plots and colouring.



        
        subGenes = volcanoPlot(dataPath, file, pVal=True, plot=False)
        if (len(subGenes) <= 1):
            print('No subgenes were found, continuing with next comparison\n\n')
            continue
        tempDF = pd.read_csv(file)
        tempDF.columns = [x.replace('.',' ') for x in tempDF.columns]
        # heatMapData = QCData.loc[subGenes,relevantAOIs] # Use EdgeR Input data for heatmaps
        heatMapData = tempDF.loc[subGenes,relevantAOIs] # Use EdgeR model output data for heatmaps

        print(heatMapData.shape)
        # print(heatMapData)


        fig, axes = plt.subplots(3, 3, figsize=(len(relevantAOIs)*0.27+3.5, len(subGenes)*0.27+3),width_ratios=[4,len(relevantAOIs),1], height_ratios=[4,2,len(subGenes)])
        ## Generate sample dendrogram

        if ((heatMapData.shape[1])>2):
            # dendroModel = get_AggloModel('EuclideanWard_T', heatMapData)
            # dendroModel = get_AggloModel('CosineWard_T', heatMapData)


            dendroModel = AgglomerativeClustering(n_clusters=None, 
                                    # metric='euclidean', 
                                    metric='cosine', 
                                    memory=None, 
                                    connectivity=kneighbors_graph(heatMapData.T,2), 
                                    # connectivity=sklearn.neighbors.kneighbors_graph(heatMapData.T,2), 
                                    # connectivity=sklearn.neighbors.kneighbors_graph(1 - sklearn.metrics.pairwise.cosine_distances(heatMapData.T),2), 
                                    compute_full_tree=True, 
                                    linkage='complete', 
                                    distance_threshold=0.1, 
                                    compute_distances=True
                                   )

        else:
            dendroModel = get_AggloModel('EuclideanWard_None', heatMapData)

        model = dendroModel.fit(heatMapData.T)
        labels = plot_dendrogram(model, 
                                 truncate_mode=None, 
                                 p=5, 
                                 ax=axes[0][1])
        AOINamesDendro = [heatMapData.columns[int(x)] for x in labels]

        
        ## Generate probe dendrogram
        if ((heatMapData.shape[0])>2):
            dendroModel = get_AggloModel('EuclideanWard', heatMapData)
        else:
            dendroModel = get_AggloModel('EuclideanWard_None', heatMapData)

        model = dendroModel.fit(heatMapData)
        labels = plot_dendrogram(model, 
                                 truncate_mode=None, 
                                 p=5, 
                                 orientation = 'left',
                                 ax=axes[2][0])
        probeNamesDendro = [heatMapData.index[int(x)] for x in labels][::-1]

        axes[0][0].axis('off')
        axes[1][0].axis('off')
        axes[0][2].axis('off')
        axes[1][2].axis('off')
        
        axes[0][1].tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False) 
        axes[1][1].tick_params(left = False, right = True, labelright = True, labelbottom = False, top = False, bottom = False) 
        axes[2][0].tick_params(left = False, right = False, labelleft = False, labelright = False, labelbottom = False, bottom = False) 

        # factors=['Histo_Stage','Segment_Name']
        # varLookup={'IA':0.1,'IIA':0.2,'IIB':0.33,'IV':0.4,'Segment1':0.65,'Segment2':0.85}
        # factors=['Obese','arthritis']
        # varLookup={'NonObese':0.1,'Obese':0.2,'Epi':0.33, 'Sub':0.4, 'Full':0.5, 'NOA':0.65,'OA':0.85}
        # factors=['Broad_classification','Tissue','Cyst_Size']
        # varLookup={'Primary':0.15,'Secondary':0.85, 'purple':0.2, 'green':0.33 , 'mixed':0.4, 'large':0.5, 'small':0.65, 'na':0.95}
        axes[1][1].matshow(get_factor_colours(AOINamesDendro, factors, varLookup, sampleInfo), 
                           cmap = 'gist_rainbow', 
                           aspect='auto', 
                           vmin=0, 
                           vmax=1)
        axes[1][1].yaxis.tick_right()
        axes[1][1].set_yticks(np.linspace(0, len(factors)-1, len(factors)), factors)
        axes[1][1].set_xticks([])

        axes[0][2].matshow([[varLookup[x]] for x in fileNameFields], 
                           cmap = 'gist_rainbow', 
                           aspect='auto', 
                           vmin=0, 
                           vmax=1)
        
        axes[2][1].matshow(np.log2(heatMapData.loc[probeNamesDendro,AOINamesDendro]), cmap = 'coolwarm', aspect='auto')
        axes[2][1].xaxis.tick_bottom()
        axes[2][1].yaxis.tick_right()
        # axes[2][1].set_xticklabels(AOINamesDendro)
        # axes[2][1].set_xticks(np.linspace(0, len(AOINamesDendro)-1, len(AOINamesDendro)), ['AOI_' + x[22:-9] for x in AOINamesDendro], rotation = 45, ha='right')
        axes[2][1].set_xticks(np.linspace(0, len(AOINamesDendro)-1, len(AOINamesDendro)), ['AOI_' + x[22:] for x in AOINamesDendro], rotation = 45, ha='right')
        axes[2][1].set_yticks(np.linspace(0, len(probeNamesDendro)-1, len(probeNamesDendro)), probeNamesDendro)

        norm = Normalize(vmin=np.log2(heatMapData.values.max()), vmax=np.log2(heatMapData.values.max()), clip=False)
        fig.colorbar(ScalarMappable(norm=norm, cmap='coolwarm'), cax=axes[2][2])
        
        fig.suptitle(fileName[:-4])
        plt.tight_layout()
        plt.savefig(file[:-4] + '_Heatmap.svg')
        plt.show()
        # break


In [ ]:
factors = list(set(factorLookup.values()))

In [ ]:
np.arange(0.1,0.9,0.8/len(factors))

In [ ]:
factors

In [ ]:
vars

In [ ]:
varLookup = zip


In [ ]:
dict(zip(factors, np.arange(0.1,0.9,0.8/len(factors))))

In [ ]:
# ToDo : Add x ticks to sample plot
# ToDo : Add legend for sample ID plot
# ToDo : Add legend for heatmap
# ToDo : automate extraction of factor names and variables

# ToDo : Use a lookup dictionary to abbreviate/shorten probe names. OR, work out a better spacing method to accommodate long probe names without creating so much white space.

In [ ]:
break

In [ ]:
sampleInfo.index

In [ ]:
import sklearn as sklearn

In [ ]:
knn = sklearn.neighbors.kneighbors_graph(heatMapData,2)
# knn = sklearn.neighbors.kneighbors_graph(heatMapData.T,2)

In [ ]:
knn.A

In [ ]:
plt.matshow(knn.A)

In [ ]:
heatMapData

In [ ]:


result = 1 - sklearn.metrics.pairwise.cosine_distances(heatMapData)

# result = 1 - spatial.distance.cosine(heatMapData)



In [ ]:
result

In [ ]:
plt.matshow(result)

In [ ]:
knnCosine = sklearn.neighbors.kneighbors_graph(result,2)

In [ ]:
knnCosine.A

In [ ]:
plt.matshow(knnCosine.A)

In [ ]:
break

In [ ]:
def calc_nonZero_mean(tempData, cols):
    tempData = tempData.loc[:,cols]
    seriesList=[]
    for factor in tempData.index:
        theseVals = tempData.loc[factor,:]
        byDict = sampleInfoTemp.loc['TMA_Core',cols].to_dict()    
        nonZeroDict = {}
        for k,v in byDict.items():
            thisVal = theseVals[k]
            if thisVal == 0:
                pass
            else:
                try:
                    nonZeroDict[v].append(thisVal)
                except KeyError:
                    nonZeroDict[v] = [thisVal]
        for k,v in byDict.items():
            if v in nonZeroDict.keys():
                nonZeroDict[v] = np.mean(nonZeroDict[v])
            else:
                nonZeroDict[v] = 0
        seriesList.append(pd.Series(nonZeroDict, name=theseVals.name))
    return(pd.DataFrame(seriesList))

In [ ]:
def expression_plot(thisData):
    figure = plt.figure(figsize=(20,10))
    figure.subplots_adjust(bottom=0.35)
    
    suffix = ['NOb_NOA', 'NOb_OA', 'Ob_NOA', 'Ob_OA']
    suffix = ['Normal_BMI', 'Normal_BMI_OA', 'Obese_BMI', 'Obese_BMI_OA']
    suffix = ['Obese_BMI', 'Obese_BMI_OA']
#     print(pd.DataFrame(thisData))
    # print('thisData')
    # print(thisData)
    
    # for p, cols in enumerate([NOb_NOA_Cols, NOb_OA_Cols, Ob_NOA_Cols, Ob_OA_Cols]):
    for p, cols in enumerate([Ob_NOA_Cols, Ob_OA_Cols]):
        tempData = calc_nonZero_mean(thisData, cols)
        pos = []
        for n in range(len(thisData.index)):
            # pos.append((n*4)+p+1)
            pos.append((n*3)+p+1)
        plt.boxplot(tempData.T, sym='-', labels=tempData.index+'_'+suffix[p], positions=pos)
#         plt.boxplot(tempData.T, sym='-', labels='FOXP3'+'_'+suffix[p], positions=pos)

        for i,j in enumerate(tempData.index):
            

            y = tempData.loc[j]
            colours = [colourList[0] if v in AOIDict['NOb_NOA'] else colourList[1] if v in AOIDict['NOb_OA'] else colourList[2] if v in AOIDict['Ob_NOA'] else colourList[3] for v in y.index]
            y = y.values
            # x = np.random.normal(1+p+(i*4), 0.1, len(y))
            x = np.random.normal(1+p+(i*3), 0.1, len(y))
            for i in range(len(x)): 
                plt.plot(x[i], y[i], c=colours[i], marker='.')
        plt.xticks(rotation = 90)
        # plt.xlabel=list(endogNorm.index[startIdx:endIdx])
    #     plt.xlabel=['CD80', 'CD66b', 'PD-L2', 'GITR', 'Phospho-GSK3B (S9)']

    plt.title('Normalised Probe Values (Log2 transformed)', size=36)
    plt.ylabel('Log2 probe value', size=24)
    plt.tight_layout(h_pad=2)
    plt.savefig(file[:-4] + '_Folicles.svg')
    plt.show()
    plt.close()
    

In [ ]:
os.getcwd()

In [ ]:
TB = ['CD20','PD-1', 'CD3', 'CD4', 'CD8', 'CTLA4', 'CD45RO', 'FOXP3', '4-1BB', 'LAG3', 'Tim-3', 'GITR', 'CD127', 'CD25', 'CD27']

# KeepNames = ['IP_TMA_OA_2022_07_28_036_Full.ROI',
#  # 'IP_TMA_OA_2022_07_28_073_Full.ROI',
#  # 'IP_TMA_OA_2022_07_28_074_Full.ROI',
#  'IP_TMA_OA_2022_07_28_042_Full.ROI',
#  'IP_TMA_OA_2022_07_28_051_Full.ROI',
#  'IP_TMA_OA_2022_07_28_053_Full.ROI',
#  'IP_TMA_OA_2022_07_28_086_Full.ROI',
#  'IP_TMA_OA_2022_07_28_092_Full.ROI']


KeepNames = ['IP_TMA_OA_2022_07_28_073_Full ROI',
 'IP_TMA_OA_2022_07_28_074_Full ROI',
 'IP_TMA_OA_2022_07_28_042_Full ROI',
 'IP_TMA_OA_2022_07_28_051_Full ROI',
 'IP_TMA_OA_2022_07_28_053_Full ROI',
 'IP_TMA_OA_2022_07_28_086_Full ROI',
 'IP_TMA_OA_2022_07_28_092_Full ROI']



In [ ]:
# sampleInfo

In [ ]:
tempDF = pd.read_csv(file)
tempDF.columns = [x.replace('.',' ') for x in tempDF.columns]
sampleInfoTemp = sampleInfo.copy()
sampleInfoTemp.columns = [x.replace('.',' ') for x in sampleInfoTemp.columns]

In [ ]:
# sampleInfoTemp

In [ ]:
tempInfo = sampleInfoTemp.loc[:,KeepNames]

In [ ]:
# tempInfo

In [ ]:
NOb = tempInfo.loc[:,sampleInfoTemp.loc['Obese'] == 'NonObese']
Ob = tempInfo.loc[:,sampleInfoTemp.loc['Obese'] == 'Obese']

NOb_NOA_Cols = NOb.loc[:,NOb.loc['arthritis'] == 'NOA'].columns
NOb_OA_Cols = NOb.loc[:,NOb.loc['arthritis'] == 'OA'].columns

Ob_NOA_Cols = Ob.loc[:,Ob.loc['arthritis'] == 'NOA'].columns
Ob_OA_Cols = Ob.loc[:,Ob.loc['arthritis'] == 'OA'].columns

In [ ]:
AOIDict = {}
suffix = ['NOb_NOA', 'NOb_OA', 'Ob_NOA', 'Ob_OA']
for i, cols in enumerate([NOb_NOA_Cols, NOb_OA_Cols, Ob_NOA_Cols, Ob_OA_Cols]):
    AOIDict[suffix[i]] = list(sampleInfoTemp.loc['TMA_Core',cols].values)

In [ ]:
colourList = [[   1, 0.5019607843137255, 0.0],
              [0.3333333333333333, 0.6274509803921569,0.984313725490196],
              [0.7215686274509804,0.33725490196078434,0.8431372549019608],
              [0,1, 0]]


In [ ]:
tempDF = tempDF.loc[:,tempDF.columns[7:]]

In [ ]:
# tempDF.loc[TB, KeepNames]

In [ ]:
# sampleInfoTemp.loc[['Obese','arthritis'],KeepNames]

In [ ]:
# tempDF

In [ ]:
# tempDF.loc[TB, KeepNames]

In [ ]:
expression_plot(np.log2(tempDF.loc[TB, KeepNames]))

In [ ]:
tempDF

In [ ]:
break

In [ ]:
# dataPath = '../../../Nanostring/projects/.../DSP_Protein_Data/'
dataPath = configDict['rootDir']
data = pd.read_csv(os.path.join(dataPath,'HK_Geo_Mean_Normalised.csv'), index_col = 0)
probeFilter = pd.read_csv(os.path.join(dataPath,'Probe_Filter.csv'), index_col = 0)
sampleInfo = pd.read_csv(os.path.join(dataPath,'Sample_Info.csv'), index_col = 0)

# dataPath = '../../../Nanostring/projects/.../EdgeR/EdgeR_normData.tsv'
dataPath = os.path.join(configDict['rootDir'], normPath, normFile)

data = pd.read_csv(dataPath, index_col = 0)

# dataPath = '../../../Nanostring/projects/.../DSP_Protein_Data/'
dataPath = configDict['rootDir']


In [ ]:
data

In [ ]:
sampleInfo.columns

In [ ]:
wb = load_workbook(os.path.join(dataPath,'Annotation template file-1a_wells_02.xlsx'))

print(wb.sheetnames)



In [ ]:
ws = wb['Annotation template']

segments = [[y.value for y in x] for x in ws[ws.calculate_dimension()]]
df = pd.DataFrame(segments)


rowLabels = df.iloc[1:,0]
colLabels = df.iloc[0,1:]
annotations = df.iloc[1:,1:]
rowLabels += '_'
rowLabels += df.iloc[1:,1]
rowLabels += '_Full ROI'



In [ ]:
rowLabels

In [ ]:
colLabels

In [ ]:
annotations.values

In [ ]:
sampleAnnotations = pd.DataFrame(annotations.values, index=rowLabels, columns=colLabels)

# sampleAnnotations = sampleAnnotations.T
# sampleAnnotations.set_index(0, drop=True, inplace=True)
# sampleAnnotations = sampleAnnotations.T
# sampleAnnotations.set_index('Scan name', drop=True, inplace=True)
# sampleAnnotations = sampleAnnotations.T



In [ ]:
sampleAnnotations

In [ ]:
sampleAnnotations = sampleAnnotations.join(sampleInfo.T,lsuffix='Drop').T

In [ ]:
sampleAnnotations.drop(labels=[x for x in sampleAnnotations.index if x.endswith('Drop')], inplace=True)

In [ ]:
sampleInfo = sampleAnnotations

In [ ]:
sampleInfo

In [ ]:
def standardize_data(arr):
         
    '''
    This function standardize an array, its substracts mean value, 
    and then divide the standard deviation.
    
    param 1: array 
    return: standardized array
    '''    
    rows, columns = arr.shape
    
    standardizedArray = np.zeros(shape=(rows, columns))
    tempArray = np.zeros(rows)
    
    for column in range(columns):
        
        mean = np.mean(X[:,column])
        std = np.std(X[:,column])
        tempArray = np.empty(0)
        
        for element in X[:,column]:
            
            tempArray = np.append(tempArray, ((element - mean) / std))
 
        standardizedArray[:,column] = tempArray
    
    return standardizedArray

In [ ]:
# Standardizing data

### I'm not sure that the transpose is what i want here. The data is the wrong shape and pc's seems to be being calculated for proteins instead of samples
X = endogNorm.transpose().values
# X = endogNorm.values
### Try to get pc's for samples
# X = endogNorm.values

## ??? With transpose makes PCA for effect of variables on smaples, without makes PCA for effects of variables on protein expression levels
###^^^ Maybe the other way round?



X_cols = endogNorm.columns
print(X_cols.shape)
y = endogNorm.index
print(y.shape)
X = standardize_data(X)
print(X.shape)


In [ ]:
# Calculating the covariance matrix

covariance_matrix = np.cov(X.T)
# covariance_matrix = np.cov(X)



print(covariance_matrix.shape)


In [ ]:
# Using np.linalg.eig function

eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)
print("Eigenvector: \n",eigen_vectors,"\n")
print("Eigenvalues: \n", eigen_values, "\n")
print(eigen_vectors.shape)

In [ ]:
eigenDF = pd.DataFrame(eigen_vectors, index=[endogNorm.index], columns=[endogNorm.index])
# eigenDF = pd.DataFrame(eigen_vectors, index=[endogNorm.columns], columns=[endogNorm.columns])

In [ ]:
# Calculating the explained variance on each of components


variance_explained = []
for i in eigen_values:
     variance_explained.append((i/sum(eigen_values))*100)
        
print(variance_explained)

In [ ]:
# Identifying components that explain at least 95%

cumulative_variance_explained = np.cumsum(variance_explained)
print(cumulative_variance_explained)


In [ ]:
cumulative_variance_explained = [np.float64(x) for x in cumulative_variance_explained]

In [ ]:
cumulative_variance_explained[0].dtype

In [ ]:
# Visualizing the eigenvalues and finding the "elbow" in the graphic


sns.lineplot(x = [i for i in range(len(cumulative_variance_explained))], y=cumulative_variance_explained)
# plt.xlabel("Number of components")
# plt.ylabel("Cumulative explained variance")
# plt.title("Explained variance vs Number of components")



# ToDo:
# Add lines for 95% variance, and number of components describing at least 95% of variance



In [ ]:
# Using two first components (because those explain more than 95%)

projection_matrix = (eigen_vectors.T[:][:50]).T
print(projection_matrix)

In [ ]:
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' for x in X_cols]
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' if x.split('_')[-1] == 'Immune' else 'purple' for x in X_cols]


In [ ]:
# I dont think this gives relevant info. projection matrix needs to be combined with original data to see effects of components on patients

plt.scatter([x[0] for x in projection_matrix], [x[1] for x in projection_matrix])#, c=colours)

projection_matrix.shape

In [ ]:
plt.scatter([x[2] for x in projection_matrix], [x[1] for x in projection_matrix])#, c=colours)


In [ ]:
plt.scatter([x[2] for x in projection_matrix], [x[3] for x in projection_matrix])#, c=colours)


In [ ]:
# Getting the product of original standardized X and the eigenvectors 


X_pca = X.dot(projection_matrix)
print(X_pca)

In [ ]:
X_pca.shape

In [ ]:
sampleInfo

In [ ]:
# tagDF = pd.DataFrame( data=[x.split(',') for x in sampleInfo.loc['Segment tags']], index=sampleInfo.columns, columns=['Obese','Arth','Patellar'])

In [ ]:
# tagDF

In [ ]:
# sampleInfo = pd.concat([sampleInfo,tagDF.T])

In [ ]:
sampleInfo

In [ ]:
inforSortedIndex = sampleInfo.sort_values(by=['Obese','arthritis','TMA_Core'], axis=1).columns

In [ ]:
### Add umap / tSNE analysis here